In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# 1. بارگذاری داده‌ها
# -----------------------------
btc = pd.read_csv(r'C:\Users\amirs\OneDrive\Desktop\myAlgoCode\detector\BTCUSDT_1m_1000d.csv')
eth = pd.read_csv(r'C:\Users\amirs\OneDrive\Desktop\myAlgoCode\detector\ETHUSDT_1m_1000d.csv')

btc_price = btc['close']
eth_price = eth['close']

# -----------------------------
# 2. محاسبه سری z و رولینگ
# -----------------------------

spread = btc_price - eth_price
window_size = 10
rolling_mean = spread.rolling(60*24*window_size).mean()
rolling_std  = spread.rolling(60*24*window_size).std()

k = 1
upper_bound = rolling_mean + k * rolling_std
lower_bound = rolling_mean - k * rolling_std
zt = spread 

# -----------------------------
# 3. شبیه‌سازی معاملات
# -----------------------------
initial_capital = 1000
btc_units = 500 / btc_price.iloc[0]
eth_units = 500 / eth_price.iloc[0]

portfolio_values = []
strategy_values  = []
trade_log = []   # ثبت معاملات

trade_size = 200   # دلار
fee_rate = 0.001   # 0.1% کارمزد

for t in range(len(spread)):
    # ارزش پرتفوی Buy&Hold
    bh_btc = (500 / btc_price.iloc[0]) * btc_price.iloc[t]
    bh_eth = (500 / eth_price.iloc[0]) * eth_price.iloc[t]
    portfolio_values.append((bh_btc, bh_eth))
    
    # محاسبه ارزش فعلی استراتژی
    total_value = btc_units * btc_price.iloc[t] + eth_units * eth_price.iloc[t]
    
    if not np.isnan(upper_bound.iloc[t]):
        if zt.iloc[t] > upper_bound.iloc[t]:
            # BTC گرونه → Sell BTC, Buy ETH
            btc_value = btc_units * btc_price.iloc[t]
            if btc_value > 0:
                sell_value = min(trade_size, btc_value)
                fee = sell_value * fee_rate
                sell_value_after_fee = sell_value - fee
                btc_units -= sell_value / btc_price.iloc[t]
                eth_units += sell_value_after_fee / eth_price.iloc[t]
                trade_log.append((t, "Sell BTC, Buy ETH", total_value))

        elif zt.iloc[t] < lower_bound.iloc[t]:
            # BTC ارزونه → Buy BTC, Sell ETH
            eth_value = eth_units * eth_price.iloc[t]
            if eth_value > 0:
                sell_value = min(trade_size, eth_value)
                fee = sell_value * fee_rate
                sell_value_after_fee = sell_value - fee
                eth_units -= sell_value / eth_price.iloc[t]
                btc_units += sell_value_after_fee / btc_price.iloc[t]
                trade_log.append((t, "Buy BTC, Sell ETH", total_value))
    
    strat_value = btc_units * btc_price.iloc[t] + eth_units * eth_price.iloc[t]
    strategy_values.append(strat_value)

# -----------------------------
# 4. محاسبه ارزش پرتفوی‌ها
# -----------------------------
bh_btc_value = [x[0] for x in portfolio_values]
bh_eth_value = [x[1] for x in portfolio_values]
bh_equal     = [x[0]+x[1] for x in portfolio_values]
strat_value  = strategy_values

# -----------------------------
# 5. تحلیل معاملات
# -----------------------------
trade_count = len(trade_log)
if trade_count > 1:
    trade_indices = [t[0] for t in trade_log]
    trade_intervals = np.diff(trade_indices)
    avg_interval = np.mean(trade_intervals)
    min_interval = np.min(trade_intervals)
    max_interval = np.max(trade_intervals)
    first_trade = trade_indices[0]
    last_trade  = trade_indices[-1]
else:
    avg_interval = min_interval = max_interval = None
    first_trade = last_trade = None

# -----------------------------
# 6. پرینت نتایج
# -----------------------------
final_btc = bh_btc_value[-1]
final_eth = bh_eth_value[-1]
final_equal = bh_equal[-1]
final_strat = strat_value[-1]

print("========== 📊 Results ==========")
print(f"Initial Capital: ${initial_capital}")
print(f"Final Strategy Value: ${final_strat:.2f}")
print(f"Total Trades: {trade_count}")

if trade_count > 1:
    print(f"Avg trade interval: {avg_interval:.1f} minutes")
    print(f"Min trade interval: {min_interval} minutes")
    print(f"Max trade interval: {max_interval} minutes")
    print(f"First trade index: {first_trade}")
    print(f"Last trade index : {last_trade}")
else:
    print("No or only one trade executed!")

print("------ Buy & Hold ------")
print(f"BTC final value : ${final_btc:.2f}  | Return: {(final_btc/500 - 1)*100:.2f}%")
print(f"ETH final value : ${final_eth:.2f}  | Return: {(final_eth/500 - 1)*100:.2f}%")
print(f"50/50 Portfolio : ${final_equal:.2f} | Return: {(final_equal/1000 - 1)*100:.2f}%")
print("------ Strategy ------")
print(f"Pair Trading     : ${final_strat:.2f} | Return: {(final_strat/1000 - 1)*100:.2f}%")

# -----------------------------
# 7. رسم نمودار
# -----------------------------
plt.figure(figsize=(12,6))
plt.plot(bh_btc_value, label='Buy & Hold BTC', alpha=0.7)
plt.plot(bh_eth_value, label='Buy & Hold ETH', alpha=0.7)
plt.plot(bh_equal, label='Buy & Hold (50/50)', alpha=0.7)
plt.plot(strat_value, label='Pair Trading Strategy (+Fees)', linewidth=2, color='black')

plt.scatter([t[0] for t in trade_log], 
            [strat_value[i] for i in [t[0] for t in trade_log]],
            marker='x', color='red', label='Trades')

plt.title("Portfolio Value Comparison (with trade analysis)")
plt.xlabel("Time (minutes)")
plt.ylabel("Portfolio Value ($)")
plt.legend()
plt.grid(True)
plt.show()
